# What fraction of the basic reproduction number "R" of COVID-19 in The Netherlands is explained by the number of positive tests in the municipality of Amsterdam?
## A study of correlation using daily Dutch COVID-19 data of the National Institute for Public Health and the Environment (RIVM)

Used data: 
- https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
- https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427

In [1]:
from datetime import datetime
from IPython.display import display
import pandas as pd
from pandas.tseries.frequencies import to_offset
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Span, Label, ColumnDataSource, Arrow, NormalHead
output_notebook()

COLORS=('#7570b3', '#d95f02', '#1b9e77')

Loading BokehJS ...

In [2]:
def load_data():
    # Loading and describing the RIVM data "cases per municipality per day"
    cases_per_municipality_per_day = pd.read_csv("COVID-19_aantallen_gemeente_per_dag.csv", sep=";")
    cases_per_municipality_per_day['Date_of_publication'] = pd.to_datetime(cases_per_municipality_per_day['Date_of_publication'], 
                                                                      format='%Y-%m-%d')
    cases_per_municipality_per_day = cases_per_municipality_per_day.set_index('Date_of_publication').sort_index()
    print(f"Total number of COVID-19 cases: {cases_per_municipality_per_day.Total_reported.sum()}")
    print(f"Total number of COVID-19 hospital admissions: {cases_per_municipality_per_day.Hospital_admission.sum()}")
    print(f"Total number of COVID-19 deaths: {cases_per_municipality_per_day.Deceased.sum()}")
    print(f"Number of rows in cases_per_municipality_per_day: {len(cases_per_municipality_per_day.index)}")

    # Creating a subset of only statistics about the municipality of Amsterdam
    amsterdam_cases_per_day = cases_per_municipality_per_day[cases_per_municipality_per_day.Municipality_name == 'Amsterdam']
    total_cases_without_amsterdam_per_day = cases_per_municipality_per_day[cases_per_municipality_per_day.Municipality_name != 'Amsterdam']
    # display(amsterdam_cases_per_day)
    print(f"Number of rows in amsterdam_cases_per_day: {len(amsterdam_cases_per_day.index)}")

    # Loading and describing the RIVM data "COVID-19 reproduction number"
    R_per_day = pd.read_json('COVID-19_reproductiegetal.json')
    R_per_day = R_per_day.set_index('Date').sort_index()
    # display(R_per_day)
    return cases_per_municipality_per_day, amsterdam_cases_per_day, total_cases_without_amsterdam_per_day, R_per_day

In [3]:
cases_per_municipality_per_day, amsterdam_cases_per_day, total_cases_without_amsterdam_per_day, R_per_day = load_data()

Total number of COVID-19 cases: 1051965
Total number of COVID-19 hospital admissions: 23761
Total number of COVID-19 deaths: 15200
Number of rows in cases_per_municipality_per_day: 136080
Number of rows in amsterdam_cases_per_day: 720


In [4]:
 def plot_reproduction_number(R_per_day):
    p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime",
               title="Reproduction number 'R' per day in the Netherlands",
               x_axis_label="Date", y_axis_label="Reproduction number", 
               toolbar_location=None, tools="",
               y_range=(0, 5))

    p.varea(x=R_per_day.index,
            y1=R_per_day.Rt_low,
            y2=R_per_day.Rt_up, color='#BDBDBD', legend_label="95% Confidence band")

    p.line(R_per_day.index, R_per_day.Rt_avg, color=COLORS[1], 
           legend_label="R average")

    hline = Span(location=1, dimension='width', line_color='grey', 
                 line_width=1, line_dash='dashed')
    p.add_layout(hline)

    add_calculation_R_change_annotation(p)

    p.x_range.start = R_per_day.index.min()
    p.x_range.end = R_per_day.index.max()
    p.legend.location = 'top_right'

    show(p)

In [5]:
def add_calculation_R_change_annotation(p):
    switch_in_methods_date = datetime.strptime('2020-06-12', '%Y-%m-%d').date()
    vline = Span(location=switch_in_methods_date, dimension='height', line_color=COLORS[0], 
                 line_width=1, line_dash='dashed')
    p.add_layout(vline)
    label1 = Label(x=switch_in_methods_date, y=4.8, x_offset = 2,
                  text='Day the RIVM switched the calculation of R', text_font_size='14px')
    label2 = Label(x=switch_in_methods_date, y=4.7, x_offset = 2,
                  text='from based on hospitalisations to positive tests', text_font_size='14px')
    p.add_layout(label1)
    p.add_layout(label2)

In [6]:
plot_reproduction_number(R_per_day)

### Figure 1: Reproduction number 'R' per day in the Netherlands
#### FIGURE CAPTION INFORMATION GOES HERE (TODO)

In [7]:
def plot_cases_netherlands_amsterdam(total_cases_without_amsterdam, amsterdam_cases):
    total_cases = total_cases_without_amsterdam.groupby(total_cases_without_amsterdam.index).sum()
    
    amsterdam_total_cases = amsterdam_cases.groupby(amsterdam_cases.index).sum()
    
    source = ColumnDataSource(data=dict(
    x=total_cases.index,
    y1=amsterdam_total_cases.Total_reported,
    y2=total_cases.Total_reported,
    ))
    
    max_y_range = total_cases.Total_reported.max() + total_cases.Total_reported.max()/10
    
    p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime",
               title="Total number of new COVID-19 cases per day in the Netherlands",
               x_axis_label="Date", y_axis_label="Total new COVID-19 cases", 
               toolbar_location=None, tools="", y_range=(0, max_y_range))

    p.varea_stack(['y1', 'y2'], x='x', color=("grey", "#BDBDBD"),
                  legend_label=("Amsterdam", "The Netherlands"), source=source)

    # Data from CBS 2020
    population_of_amsterdam = 872779
    population_of_the_netherlands = 17407585
    
    p.line(x=total_cases.index, 
           y=total_cases.Total_reported*(population_of_amsterdam/population_of_the_netherlands), 
           color=COLORS[1], line_width=2, 
           legend_label='Expected fraction of Dutch cases to be in Amsterdam based on the population')
    
    add_COVID_distribution_annotation(p, total_cases)
    
    p.x_range.start = total_cases.index.min()
    p.x_range.end = total_cases.index.max()
    p.legend.location = 'top_left'

    show(p)

In [8]:
def add_COVID_distribution_annotation(p, total_cases):
    highlighted_date_start = datetime.strptime('2020-09-01', '%Y-%m-%d').date()
    highlighted_date_end = datetime.strptime('2020-12-01', '%Y-%m-%d').date()
    
    label1 = Label(x=highlighted_date_start, y=3300, x_offset = -400, y_offset=0, text_color='black',
                  text='If COVID-19 cases are uniformly distributed over the Netherlands', text_font_size='14px')
    label2 = Label(x=highlighted_date_start, y=3000, x_offset = -400, y_offset=0, text_color='black',
                  text='you would expect the orange line to follow the dark grey area', text_font_size='14px')
    
    arrow = Arrow(end=NormalHead(line_color="black", line_width=1), x_start=highlighted_date_start, 
                  y_start=3000, x_end=highlighted_date_end, y_end=500)
    
    p.add_layout(label1)
    p.add_layout(label2)
    p.add_layout(arrow)

In [9]:
plot_cases_netherlands_amsterdam(total_cases_without_amsterdam_per_day, amsterdam_cases_per_day)

## Figure 2: Total number of new COVID-19 cases per day in the Netherlands
#### FIGURE CAPTION INFORMATION GOES HERE (TODO)

# TODO: create same plot as above but weekly instead of daily

In [10]:
amsterdam_cases_per_week = amsterdam_cases_per_day.resample('W', label='left').sum()
amsterdam_cases_per_week.index = amsterdam_cases_per_week.index + to_offset("1D")
cases_per_municipality_per_week = total_cases_without_amsterdam_per_day.resample('W', label='left').sum()
cases_per_municipality_per_week.index = cases_per_municipality_per_week.index + to_offset("1D")

plot_cases_netherlands_amsterdam(cases_per_municipality_per_week, amsterdam_cases_per_week)